# Notebook 2

# Ejercicio 1. 

¿De qué podría servir esta base de datos? ¿Cuál era su aplicación probable? 
¿Cuál sería una adaptación posible más moderna?

Podría servir para una tienda de alquiler de videos, como antiguamente lo era blockbuster.
La aplicación de la bd era buscar películas mediante sus características para el posterior arriendo de esta.
Una adaptación moderna: podría ser utilizarla para Netflix.

# Ejercicio 2.

¿Cómo importar una base de datos?

1) mysql -u USUARIO -p
   Luego se ingresa la contraseña del usuario.



2) CREATE DATABASE new_database; 
Se crea la base de datos a la que queremos importar los datos.

3) USE new_database;

4) source \home\usuario\Escritorio\archivo.sql
Se llama al archivo .sql del que deseamos importar los datos. 

Así se puede comenzar a utilizar la base de datos

# Ejercicio 3.


In [2]:
import mysql.connector
import pandas as pd
from IPython.display import display,HTML

db_connection = mysql.connector.connect(user="root",host="localhost",password="94209420")
cursor = db_connection.cursor()
cursor.execute("USE sakila")

1) A partir de la tabla "film", mostrar solamente cuáles son los títulos de pelicula disponibles.

In [3]:
df = pd.read_sql('SELECT title FROM film LIMIT 10;', con=db_connection)
display(df)

,title
0,ACADEMY DINOSAUR
1,ACE GOLDFINGER
2,ADAPTATION HOLES
3,AFFAIR PREJUDICE
4,AFRICAN EGG
5,AGENT TRUMAN
6,AIRPLANE SIERRA
7,AIRPORT POLLOCK
8,ALABAMA DEVIL
9,ALADDIN CALENDAR


2) A partir de la tabla "film", mostrar de qué años son las peliculas disponibles. NB: utilizar la instrucción DISTINCT

In [4]:
df = pd.read_sql('SELECT DISTINCT release_year FROM film LIMIT 5;', con=db_connection)
display(df)

,release_year
0,2006


3) Mostrar los títulos de pelicula en la tabla "film" que se puede arrendar por más de 4 días (rental_duration)

In [5]:
df = pd.read_sql('SELECT title, rental_duration '+'FROM film '+'WHERE rental_duration >= 4 LIMIT 5;', con=db_connection)
display(df)

,title,rental_duration
0,ACADEMY DINOSAUR,6
1,ADAPTATION HOLES,7
2,AFFAIR PREJUDICE,5
3,AFRICAN EGG,6
4,AIRPLANE SIERRA,6


4) En la tabla "payment", mostrar el customer_id, rental_id, amount y payment_date de todos los arriendos realizados entre las fechas 2005-07-01 y 2005-08-01

In [6]:
df = pd.read_sql('SELECT customer_id, rental_id, amount, payment_date ' + 'FROM payment '+ 'WHERE payment_date BETWEEN \"2005-07-01\" AND \"2005-08-01\" LIMIT 5;', con=db_connection)
display(df)

,customer_id,rental_id,amount,payment_date
0,1,4526,5.99,2005-07-08 03:17:05
1,1,4611,5.99,2005-07-08 07:33:56
2,1,5244,4.99,2005-07-09 13:24:07
3,1,5326,4.99,2005-07-09 16:38:01
4,1,6163,7.99,2005-07-11 10:13:46


# Ejercicio 4

1) Contar el número de clientes en la tabla "CUSTOMER"


In [7]:
df = pd.read_sql('SELECT count(*) FROM customer;', con=db_connection)
display(df)

,count(*)
0,599


2) En la tabla "RENTAL", contar cuánto arriendos se realizó por cada cliente (NB: utilizar la instucción GROUP BY y COUNT)

In [8]:
df = pd.read_sql('SELECT COUNT(rental_id), customer_id FROM rental GROUP BY customer_id LIMIT 10;', con=db_connection)
display(df)

,COUNT(rental_id),customer_id
0,32,1
1,27,2
2,26,3
3,22,4
4,38,5
5,28,6
6,33,7
7,24,8
8,23,9
9,25,10


3) ¿Qué devuelve la consulta SQL siguiente?

SELECT MAX(total) FROM (SELECT customer_id,count(*) as total FROM rental GROUP BY customer_id) as t1;

Es una subconsulta. Primero agrupa los clientes con su cantidad de rentas respectivas. 
Luego selecciona el valor máximo de rentas que se han realizado por un cliente.  

In [9]:
df = pd.read_sql('SELECT MAX(total) FROM (SELECT customer_id, count(*) as total FROM rental GROUP BY customer_id) as t1;', con=db_connection)
display(df)

,MAX(total)
0,46


4) ¿Qué es un sub-consulta?

Es una consulta anidada dentro de otra consulta.

5) En la tabla "payment", mostrar el volumen de negocio realizado por cada miembro del staff (staff_id) en total.

In [69]:
df = pd.read_sql('SELECT staff_id, SUM(amount) as total FROM payment GROUP BY staff_id;', con=db_connection)
display(df)

,staff_id,total
0,1,33489.47
1,2,33927.04


6) En una consulta, mostrar el staff_id que ha realizado más volumen de negocio el dia 2005-08-23.

In [192]:
df = pd.read_sql('SELECT MAX(total) FROM (SELECT staff_id, SUM(amount) as total '
                 + 'FROM payment WHERE payment_date LIKE "2005-08-23%" '
                 + 'GROUP BY staff_id) as temd ;', con=db_connection)
display(df)

,MAX(total)
0,1391.69


# Ejercicio 5

1) En las peliculas "Film" y "Language", mostrar el título de la pelicula y el nombre de su idioma. (NB: utilizar una operación de INNER JOIN)

In [13]:
df = pd.read_sql('SELECT title, name FROM film INNER JOIN language ON film.language_id = language.language_id LIMIT 10;', con=db_connection)
display(df)

,title,name
0,ACADEMY DINOSAUR,English
1,ACE GOLDFINGER,English
2,ADAPTATION HOLES,English
3,AFFAIR PREJUDICE,English
4,AFRICAN EGG,English
5,AGENT TRUMAN,English
6,AIRPLANE SIERRA,English
7,AIRPORT POLLOCK,English
8,ALABAMA DEVIL,English
9,ALADDIN CALENDAR,English


2) Mostrar los nombres y apellidos de los clientes que han arrendado la pelicula "CHICAGO NORTH"

In [18]:
df = pd.read_sql('SELECT first_name, last_name FROM customer INNER JOIN rental ON customer.customer_id = rental.customer_id' 
                 + ' INNER JOIN inventory ON rental.inventory_id = inventory.inventory_id'
                 + ' INNER JOIN film ON inventory.inventory_id = film.film_id'
                 + ' WHERE title = "CHICAGO NORTH"', con=db_connection)
display(df)

,first_name,last_name
0,THEODORE,CULP
1,ALVIN,DELOACH
2,HAROLD,MARTINO
3,DANA,HART
4,DEBBIE,REYES


3) Mostrar por cada nombres y apellidos de cliente, en qué ciudad viven (tabla Customer y City)

In [20]:
df = pd.read_sql('SELECT first_name, last_name, city FROM customer INNER JOIN address'
                 +' ON customer.address_id = address.address_id'
                 +' INNER JOIN city ON address.city_id = address.city_id LIMIT 10;', con=db_connection)
display(df)

,first_name,last_name,city
0,MARY,SMITH,A Corua (La Corua)
1,PATRICIA,JOHNSON,A Corua (La Corua)
2,LINDA,WILLIAMS,A Corua (La Corua)
3,BARBARA,JONES,A Corua (La Corua)
4,ELIZABETH,BROWN,A Corua (La Corua)
5,JENNIFER,DAVIS,A Corua (La Corua)
6,MARIA,MILLER,A Corua (La Corua)
7,SUSAN,WILSON,A Corua (La Corua)
8,MARGARET,MOORE,A Corua (La Corua)
9,DOROTHY,TAYLOR,A Corua (La Corua)


4) Mostrar cuántas veces fueron arrendadas las peliculas (mostrando el título de la pelicula)

In [155]:
df = pd.read_sql('SELECT title, count(*) as cantidad FROM rental INNER JOIN inventory '
                 + 'ON rental.inventory_id = inventory.inventory_id '
                 + 'INNER JOIN film ON film.film_id = inventory.film_id GROUP BY title LIMIT 10; ' , con=db_connection)
display(df)

,title,cantidad
0,ACADEMY DINOSAUR,23
1,ACE GOLDFINGER,7
2,ADAPTATION HOLES,12
3,AFFAIR PREJUDICE,23
4,AFRICAN EGG,12
5,AGENT TRUMAN,21
6,AIRPLANE SIERRA,15
7,AIRPORT POLLOCK,18
8,ALABAMA DEVIL,12
9,ALADDIN CALENDAR,23


5) Mostrar qué pelicula generó más volumen de negocio

In [183]:
df = pd.read_sql('SELECT MAX(total) FROM (SELECT title, SUM(renta) as total FROM (SELECT title, rental_rate as renta FROM rental INNER JOIN inventory '
                 + 'ON rental.inventory_id = inventory.inventory_id '
                 + 'INNER JOIN film ON film.film_id = inventory.film_id) as tem1 GROUP BY title) tem3; ' , con=db_connection)
display(df)

,MAX(total)
0,169.66


6) Mostrar los clientes que arrendarón por más de 10 dolares de pelicula.


No entendí la pregunta